# Just a script to combine spreadsheets and make the tables

In [1]:

import pandas as pd
import os


cancer = 'ClearCellRenalCellCarcinoma'
input_dir = 'Input_RCM'
output_dir = 'Output_Data'
supp_dir = 'Required_Refs'
fig_dir = 'Output_Figures'
regLabel = 'RG2_Changes_filtered'


df = pd.read_csv(os.path.join(output_dir, f'sircle_PorMandR_{cancer}.csv'))
cols_to_keep = [c for c in df.columns if 'C3' not in c]
df = df[cols_to_keep]

/var/folders/gq/6ljhmvm1713fykdjqbl188pm0000gn/T/ipykernel_54170/562453557.py:13: DtypeWarning: Columns (1,2,353,381,383,384,388,389,583,584,585) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(output_dir, f'sircle_PorMandR_{cancer}.csv'))


In [2]:
import os
ora_dir = os.path.join(output_dir, 'ORA')
files = os.listdir(ora_dir)
df_2 = pd.DataFrame()
cluster_files = [c for c in files if 'ClusterGoSummary' in c]
all_ora = pd.DataFrame()
for c in cluster_files:
    if '.svg' not in c and '_RCM' in c:
        df_2 = pd.concat([df_2, pd.read_csv(f'{ora_dir}/{c}')])


In [3]:
import os
ora_dir = os.path.join(output_dir, 'ORA')
files = os.listdir(ora_dir)
df_3 = pd.DataFrame()
cluster_files = [c for c in files if 'ClusterGoSummary' in c]
all_ora = pd.DataFrame()
for c in cluster_files:
    if '.svg' not in c and '_RCM' not in c:
        df_t = pd.read_csv(f'{ora_dir}/{c}')
        df_t['file'] = c.replace('ClusterGoSummary_', '').replace('.csv', '')
        df_3 = pd.concat([df_3, df_t])


In [4]:
with pd.ExcelWriter('/Users/ariane/Documents/code/roundround/tables/Table3_SiRCle_ORA.xlsx') as writer:  
    df.to_excel(writer, sheet_name='SiRCle')  
    df_3.to_excel(writer, sheet_name='ORA_TvN')
    df_2.to_excel(writer, sheet_name='ORA_SiRCle')
    # Also for each of the DE analyses lets add them in here
    p_df = pd.read_csv('Input_Protein/ClearCellRenalCellCarcinoma_filtered_DA_Protein.csv')
    protein_cols = ['gene_name', 'logFC_protein', 'stat_protein', 'pvalue_protein', 
                'padj_protein', 'B_protein', 'mean_protein']
    p_df[protein_cols].to_excel(writer, sheet_name='Protein_DA')
    rna_cols = ['gene_name', 'gene_id', 'logFC_rna', 'stat_rna', 'pvalue_rna', 
                'padj_rna', 'lfcSE_rna', 'baseMean_rna', 'var_rna']
    r_df = pd.read_csv('Input_RNAseq/ClearCellRenalCellCarcinoma_filtered_DE_RNA.csv')
    r_df[rna_cols].to_excel(writer, sheet_name='RNA_DE')
    cpg_cols = ['Name', 'beta_diff', 'AveExpr', 't', 'P.Value', 'adj.P.Val', 'B', 'gene_name']
    d = pd.read_csv('Input_Methylation/ClearCellRenalCellCarcinoma_filtered_DCpG.csv')
    d[cpg_cols].to_excel(writer, sheet_name='CpG_DMC')
    pd.read_csv(f'{fig_dir}/Pathways_Protein_GSEA_MetabolicPathways.csv').to_excel(writer, sheet_name=f'Protein_TvN_metabolic')
    pd.read_csv(f'{fig_dir}/Pathways_RNA_GSEA_MetabolicPathways.csv').to_excel(writer, sheet_name=f'RNA_TvN_metabolic')
    pd.read_csv(f'{fig_dir}/Pathways_DMC_GSEA_MetabolicPathways.csv').to_excel(writer, sheet_name=f'Methylation_TvN_metabolic')


/var/folders/gq/6ljhmvm1713fykdjqbl188pm0000gn/T/ipykernel_54170/2065201414.py:15: DtypeWarning: Columns (212) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv('Input_Methylation/ClearCellRenalCellCarcinoma_filtered_DCpG.csv')


## Now create the tables also for TvN early vs late TvN

In [4]:
with pd.ExcelWriter('/Users/ariane/Documents/code/roundround/tables/Table6_ORA_EarlyLate_TvN.xlsx') as writer:
    paths = ['ORA_Early', 'ORA_Late', 'ORA_EarlyLate']
    for p in paths:
        ora_dir = os.path.join(output_dir, p)
        all_ora = pd.DataFrame()
        files = os.listdir(ora_dir)
        cluster_files = [c for c in files if 'ClusterGoSummary' in c]
        for c in cluster_files:
            if '.svg' not in c:
                df_t = pd.read_csv(f'{ora_dir}/{c}')
                df_t['Condition'] = c
                all_ora = pd.concat([all_ora, df_t])
        all_ora.to_excel(writer, sheet_name=p)
            # Also for each of the DE analyses lets add them in here
        p = p.split('_')[1]
        p_df = pd.read_csv(f'Input_Protein/ClearCellRenalCellCarcinoma_filtered_DA_Protein_{p}.csv')
        protein_cols = ['gene_name', 'logFC_protein', 'stat_protein', 'pvalue_protein', 
                    'padj_protein', 'B_protein', 'mean_protein']
        p_df[protein_cols].to_excel(writer, sheet_name=f'Protein_DA_{p}')
        rna_cols = ['gene_name', 'gene_id', 'logFC_rna', 'stat_rna', 'pvalue_rna', 
                    'padj_rna', 'lfcSE_rna', 'baseMean_rna', 'var_rna']
        r_df = pd.read_csv(f'Input_RNAseq/ClearCellRenalCellCarcinoma_filtered_DE_RNA_{p}.csv')
        r_df[rna_cols].to_excel(writer, sheet_name=f'RNA_DE_{p}')
        cpg_cols = ['Name', 'beta_diff', 'AveExpr', 't', 'P.Value', 'adj.P.Val', 'B', 'gene_name']
        d = pd.read_csv(f'Input_Methylation/ClearCellRenalCellCarcinoma_filtered_DCpG_{p}.csv')
        d[cpg_cols].to_excel(writer, sheet_name=f'CpG_DMC_{p}')
    pd.read_csv(f'{fig_dir}/Early_vs_Late_Protein_GSEA_MetabolicPathways.csv').to_excel(writer, sheet_name=f'Protein_LatevsEarly_metabolic')
    pd.read_csv(f'{fig_dir}/Early_vs_Late_RNA_GSEA_MetabolicPathways.csv').to_excel(writer, sheet_name=f'RNA_LatevsEarly_metabolic')
    pd.read_csv(f'{fig_dir}/Early_vs_Late_DMC_GSEA_MetabolicPathways.csv').to_excel(writer, sheet_name=f'Methylation_LatevsEarly_metabolic')


/var/folders/gq/6ljhmvm1713fykdjqbl188pm0000gn/T/ipykernel_52114/1631081606.py:25: DtypeWarning: Columns (212) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(f'Input_Methylation/ClearCellRenalCellCarcinoma_filtered_DCpG_{p}.csv')
/var/folders/gq/6ljhmvm1713fykdjqbl188pm0000gn/T/ipykernel_52114/1631081606.py:25: DtypeWarning: Columns (212) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(f'Input_Methylation/ClearCellRenalCellCarcinoma_filtered_DCpG_{p}.csv')
/Users/ariane/opt/miniconda3/envs/roundround/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
